In [ ]:
import uproot3 as uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
date = datetime.today().strftime('%Y%m%d')

def loadSingleFile( tfile, treename, flatenndf=False ):
    ttree = uproot.open(tfile)
    data = ttree[treename].arrays(outputtype=pd.DataFrame,flatten=flatenndf)
    return data

def loadFiles( filelist, treename1, treename2, maxfiles=100, flatenndf=False):
    data1 = pd.DataFrame()
    data2 = pd.DataFrame()
    for i,tfile in enumerate(filelist[0:maxfiles]):
        if i%10 == 0:
            print("{} files processed".format(i))
        ttree = uproot.open(tfile)
        data1 = pd.concat([data1, ttree[treename1].arrays(outputtype=pd.DataFrame,flatten=flatenndf)])
        data2 = pd.concat([data2, ttree[treename2].arrays(outputtype=pd.DataFrame,flatten=flatenndf)])
    return data1, data2

## Prepare data files

In [ ]:
### PREPARE DATA ####
RUN = "8460"
MATCHES = "../output/run{}_matched_light_tracks.txt".format(RUN)
LIGHTINFO = "../inputs/run{}_tracks_BNBMAJORITY_files.txt".format(RUN)
OUTFILE = "../output/residuals/run{}_cosmics_timing_residuals_{}.csv".format(RUN,date)
FILENAMES = [ line.strip() for line in open(LIGHTINFO, "r") ]

maxim = 100
#maxim = len(FILENAMES)

### CASE 1: New (complete) light trees 

In [ ]:
## Get the light data and combine the two cryostats
dfw, dfe = loadFiles(FILENAMES, "simpleLightAna/opflashCryoW_flashtree", "simpleLightAna/opflashCryoE_flashtree", maxim)
dfw["cryo"] = 1
dfe["cryo"] = 0
dfw.drop(columns=["multiplicity","multiplicity_right","multiplicity_left","sum_pe","sum_pe_right","sum_pe_left"],inplace=True)
dfe.drop(columns=["multiplicity","multiplicity_right","multiplicity_left","sum_pe","sum_pe_right","sum_pe_left"],inplace=True)
df = pd.concat([dfe, dfw])

del dfw
del dfe

In [ ]:
## Now match with the selected tracks
dfmatches = pd.read_csv(MATCHES)
dfmatches.rename(columns={'flashID':"flash_id"}, inplace=True)
dfmatches.set_index(["run", "event", "cryo", "flash_id"], inplace=True)
df = (df.join( dfmatches, on=["run", "event", "cryo", "flash_id"], how='inner'))
df["channel_id"] = df.pmt_y.apply( lambda x : np.arange(len(x)) )

print("Consider {} tracks".format( len(df) ) )

In [ ]:
df.head()

### CASE 2: Old (incomplete light trees)

In [ ]:
dfw, dfe = loadFiles(FILENAMES, "simpleLightAna/opflashCryoW_flashtree", "simpleLightAna/opflashCryoE_flashtree", maxim)
dfw["cryo"] = 1
dfe["cryo"] = 0
dfw.drop(columns=["multiplicity","multiplicity_right","multiplicity_left","sum_pe","sum_pe_right","sum_pe_left"],inplace=True)
dfe.drop(columns=["multiplicity","multiplicity_right","multiplicity_left","sum_pe","sum_pe_right","sum_pe_left"],inplace=True)
df = pd.concat([dfe, dfw])

del dfe
del dfw

In [ ]:
dfopw, dfope = loadFiles(FILENAMES, "simpleLightAna/opflashCryoW_ophittree", "simpleLightAna/opflashCryoE_ophittree", maxim)
dfopw["cryo"] = 1
dfope["cryo"] = 0
dfop = pd.concat([dfope, dfopw])

del dfope
del dfopw

In [ ]:
geo = loadSingleFile(FILENAMES[0], "simpleLightAna/geotree")
x = geo.pmt_x.values[0] ; y = geo.pmt_y.values[0] ; z = geo.pmt_z.values[0]
dfop["pmt_x"] = [ x[ch] for ch in dfop.channel_id.values ] 
dfop["pmt_y"] = [ y[ch] for ch in dfop.channel_id.values ] 
dfop["pmt_z"] = [ z[ch] for ch in dfop.channel_id.values ]

del geo

In [ ]:
dfop = dfop.groupby(["run", "event", "cryo","flash_id","channel_id"]).agg({ "start_time":"min", "pmt_y":"mean", "pmt_x":"mean", "pe":"sum", "pmt_z":"mean" }).reset_index()
dfop = dfop.groupby(["run", "event", "cryo","flash_id"]).agg(list)
dfop.rename(columns={'start_time':"time_pmt", "pe":"pe_pmt"}, inplace=True)
df = (df.join( dfop, on=["run", "event", "cryo", "flash_id"], how='inner'))

del dfop

In [ ]:
## Now match with the selected tracks
dfmatches = pd.read_csv(MATCHES)
dfmatches.rename(columns={'flashID':"flash_id"}, inplace=True)
dfmatches.set_index(["run", "event", "cryo", "flash_id"], inplace=True)
df = (df.join( dfmatches, on=["run", "event", "cryo", "flash_id"], how='inner'))

print("Consider {} tracks".format( len(df) ) )

In [ ]:
df.head()

## Check tracks

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 4.3))

ax[0].plot( [df.trackEndZ, df.trackStartZ], [df.trackEndY, df.trackStartY], color='black', lw=0.1 )
ax[1].plot( [df.trackEndX, df.trackStartX], [df.trackEndY, df.trackStartY], color='black', lw=0.1 )

# consider steeper angle
_sel_dir_z= (df.trackDirZ > -0.3) & (df.trackDirZ < 0.3 ) 
_sel_dir_x = (df.trackDirX > -0.1) & (df.trackDirX < 0.1 )
_seldir= _sel_dir_z
#ax[0].plot( [dfmatches[_seldir].trackEndZ, dfmatches[_seldir].trackStartZ], [dfmatches[_seldir].trackEndY, dfmatches[_seldir].trackStartY], color='red', lw=0.1 )
#ax[1].plot( [dfmatches[_seldir].trackEndX, dfmatches[_seldir].trackStartX], [dfmatches[_seldir].trackEndY, dfmatches[_seldir].trackStartY], color='red', lw=0.1 )

print( len(df), len(df[_seldir]) )

ax[0].set_ylabel("Y [cm]" )  
ax[0].set_xlabel("Z [cm]" )  
ax[1].set_ylabel("Y [cm]" )  
ax[1].set_xlabel("X [cm]" )  

plt.savefig("../figs/run()_tracks_standard_selection.png".format(RUN),dpi=100)
plt.show()

## Explode the dataframe

In [ ]:
## Explode the dataframe 
df = df.explode(["time_pmt", "pmt_x", "pmt_y", "pe_pmt", "pmt_z", "channel_id"])

## Add timing corrections (if needed!)

In [ ]:
## Import and use laser correction
## WARNING: DO NOT USE IF CORRECTIONS WERE ALREADY APPLIED AT STAGE0 
## note: laser corrections are in ns!

LASERCORR="../inputs/laser_time_corrections_20220408.csv"
lasercorr = pd.read_csv(LASERCORR).set_index(["channel_id"])
lasercorr['t_signal'] = lasercorr['t_signal']/1e3  #convert ns to us

df = df.join( lasercorr[["t_signal"]], on=["channel_id"])
df['time_pmt'] = df['time_pmt'] + df['t_signal']  ###WARNING:: CURRENLTY REMOVING LASER!!!

In [ ]:
## WARNING: adding cosmics corrections
COSMICSCORR = "../output/residuals/run8462_cosmics_timing_residuals_20230725.csv"
cosmics = pd.read_csv(COSMICSCORR).set_index("channel_id")

df = df.join( cosmics["mean_residual_ns"], on="channel_id" )
df["time_pmt"] = df["time_pmt"]-df["mean_residual_ns"]/1e3 #convert ns to us

In [ ]:
df.loc[df["pe_pmt"]>100,["event","cryo","flash_id","pmt_y","channel_id","time_pmt",'t_signal','mean_residual_ns']]

In [ ]:
df = df.drop(columns=["t_signal"])

In [ ]:
df = df.drop(columns=["mean_residual_ns"])

## Computing fit and residuals

In [ ]:
_pecut=300

_sel = df.pe_pmt > _pecut
meandf = df[_sel][["run", "event", "cryo", "flash_id", "time_pmt", "pe_pmt", "pmt_y"]].groupby(["run", "event", "cryo", "flash_id", "pmt_y"]).apply( 
    lambda x : pd.Series( {
        "mean_time" : np.mean(x.time_pmt),
        "weight_mean_time" : np.average(x.time_pmt, weights=x.pe_pmt), 
        "error_mean_time": np.std(x.time_pmt) / np.sqrt(len(x.time_pmt)),
    }) ).reset_index()

meandf = meandf.groupby(["run", "event", "cryo", "flash_id"]).agg(list)

print("PE cut leaves {} tracks".format( len(meandf) ))

In [ ]:
def getdiff( y, t):
    # max y is on top : cosmics are going towards decreasing y 
    return t[np.argmin(y)] - t[np.argmax(y)]

def fittime( y, t ):
    try:
        res= stats.linregress(y, t)
        #print( res.intercept, res.slope)
        return res.intercept,  res.slope
    except:
        return 0,0

def residuals( tobs, y, a, b ):
    return tobs -  ( a + b*y )

In [ ]:
meandf["diff_time"] = meandf.apply( lambda x : getdiff( x.pmt_y, x.mean_time ), axis=1 ) 
meandf[["intercept", "slope"]] = meandf.apply(lambda x : fittime(x.pmt_y, x.mean_time ), axis=1, result_type="expand" )

In [ ]:
meandf.head()

In [ ]:
# Putting fit back in the exploded dataframe, then compute the residual
# This should work for every channel_id

# TEST: only positive slopes
#dfg = df.join( meandf[meandf.slope<0][["intercept", "slope"]], on=["run", "event", "cryo", "flash_id"], how='inner')

# all slopes
dfg = df.join( meandf[["intercept", "slope"]], on=["run", "event", "cryo", "flash_id"], how='inner')
dfg["residuals"] = dfg.apply( lambda x : residuals(x.time_pmt, x.pmt_y, x.intercept, x.slope), axis=1 ) 

# Keep only the residuals on relevant PMT for that event
PECUT = 300
dfg = dfg[dfg.pe_pmt>PECUT]

In [ ]:
print("Using the above slopes leaves {} tracks".format( len(dfg.groupby(["run", "event", "cryo", "flash_id"])) ) )

In [ ]:
dfg.head()

## Group and save residuals

In [ ]:
us_to_ns = 1e3
thisdfg = dfg.groupby(["channel_id"]).apply(
    lambda x : pd.Series( { 
            'x': np.mean(x.pmt_x),
            'y': np.mean(x.pmt_y),
            'z': np.mean(x.pmt_z),
            'entries' : len(x.residuals), 
            'pecut' : PECUT,
            'mean_residual_ns' : np.mean(x.residuals)*us_to_ns,
            'std_residual_ns' : np.std(x.residuals)*us_to_ns,
            'emean_ns' : np.std(x.residuals)*us_to_ns/len(x.residuals)
        })).reset_index()

In [ ]:
thisdfg.head()

In [ ]:
thisdfg.to_csv(OUTFILE, index=False, float_format='%.4f')

## Looking at some residuals

In [ ]:
selected_channel = 15
residuals = dfg[dfg.channel_id==selected_channel].residuals.values

In [ ]:
fig = plt.figure(dpi=100)

lab = "Channel ID "+str(selected_channel)+"\n"
lab += "Entries: {}\n".format(len(residuals))
lab += "Mean: {:.2f} ns\n".format(np.mean(residuals*1e3))
lab += "Std: {:.2f} ns".format(np.std(residuals*1e3))

plt.hist( residuals*1e3, bins=50, range=(-10,20), histtype='step', label=lab)
#plt.ylabel("# entries", fontsize=16)
plt.xlabel("Residuals [ns]", fontsize=14)
plt.tight_layout()
plt.grid(linestyle="dashed", alpha=0.5)
plt.legend(fontsize=12)
#plt.savefig("../figs/run8462_channel_{}_residuals.png".format(selected_channel),dpi=100)
plt.show()


In [ ]:
# Plotting full distribution 
fig = plt.figure(figsize=(10,5),dpi=300)

rmin=-10
rmax=10
r=(rmin,rmax)
s=0.5
b=int((rmax-rmin)/s)

lab = "Run {}\nMean: {:.2f} ns\nStd: {:.2f} ns".format(RUN,np.mean(thisdfg.mean_residual_ns),np.std(thisdfg.mean_residual_ns))

plt.hist(thisdfg.mean_residual_ns, bins=b, linewidth=2, range=r, histtype="step", label=lab)

plt.xlabel("Time residual [ns]")
plt.ylabel("# PMTs")
plt.legend()
plt.grid(linestyle="dashed", alpha=0.5)
plt.savefig("../figs/run{}_residuals_base&laser.png".format(RUN),dpi=100)
plt.show()

## Looking at slope

In [ ]:
slopes = meandf["slope"].values
len(slopes)

In [ ]:
fig = plt.figure(figsize=(8,4),dpi=100)

plt.hist( slopes*1e3, bins=50, range=(-0.1,0.075), histtype='step')
plt.ylabel("# Flash-Track matches", fontsize=14)
plt.xlabel("Fitted slope [cm ns$^{-1}$]", fontsize=14)

plt.axvline(x=0.,color="red",linestyle="dotted")

#plt.tight_layout()
plt.grid(linestyle="dashed",alpha=0.5)
#plt.legend()
plt.savefig("../figs/run()_slope_distribution.png".format(RUN))
plt.show()
